<a href="https://colab.research.google.com/github/uni-3/jupyter-notebooks/blob/master/azure_speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# refer here https://github.com/Azure-Samples/cognitive-services-speech-sdk/blob/master/quickstart/python/quickstart.ipynb
!pip install azure-cognitiveservices-speech

In [0]:
import time
import azure.cognitiveservices.speech as speechsdk

In [0]:
# google drive内のをディレクトリをマウント
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls ./gdrive/'My Drive'/"Colab Notebooks"

In [0]:
path = './gdrive/My Drive/Colab Notebooks/'
filename = path + 'TBS_2017072310_cut_3min.wav'
# fileの存在確認
# import os
# os.path.exists(filename)

In [0]:
#@title init api config
speech_key = "" #@param {type:"string"}
service_region = "westus"
language = 'ja-JP'

In [0]:
# recognisionResult class
# https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.recognitionresult?view=azure-python
def speech_recognize_continuous_from_file():
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region, speech_recognition_language=language)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True
    
    # get result text as list
    results = []
    def recog_cb(evt):
      print('RECOGNIZED: {}'.format(evt))
      results.append(evt.result.text)
      return results

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    #speech_recognizer.recognizing.connect(recog_cb)
    #speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.recognized.connect(recog_cb)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    # </SpeechContinuousRecognitionWithFile>
    
    return results

In [0]:
results = speech_recognize_continuous_from_file()

In [0]:
print(results)

['勉強した方がこれを早くしているところでちょっとこうした状況なんでしょうねだと思うんだよしています。そうすると変わるじゃないですか、そうだからあのやっぱり素晴らしい名を残しておきたいなと思いまして消えるわけも応援と思う危険なんですよねけないと思ってたんですよ。', 'ほら、私興味あるスタンドで取材したときに作調整方向から。', '上にんぐ特徴が聞こえてきて変動してだろうと思って泣いたんですよね。あのもう開いと思ってたのであええと思ったら削除せない。', '藤原監督がもともと pl の出身で pl で監督やられてってうつって削除性にこれで？', 'お野球部の部員の提案でビールのを継承したいということで使われたと聞いてますねはいメモを応援がそうですね。県の野球部で蘇る。すねはあさて、それでは嬉しさが監修した cd の中から一曲を聞いたときたいとおもいます。演奏はシンフォニックウィンドウ。', 'アンサンブル二十一掛け声は私出ています。川越東高等学校吹奏楽部のみなさんです。アフリカの進歩にどうぞ。', 'atf。', '', 'atf。', 'atf。', 'atf。', 'さあ改めまして経ゲストにお迎えしていますのは高校野球部ラバン応援研究科の梅つゆき子さんです。', '嫁さんのこれまでの研究結果をまとめた。五本プラバン甲子園たい研究は、文藝春秋社から税別1200円で発売中です。', 'そして不滅さんにとっては、こちらも本業ということで手軽に作れるお料理のはい今夜も終電ごはんが幻冬舎から税別1000円で発売。']


In [0]:
# save to file
with open(path + 'result.txt', 'w') as f:
  f.writelines(["%s\n" % result for result in results])

### Text Analytics Cognitive Service
https://docs.microsoft.com/en-us/azure/cognitive-services/Text-Analytics/QuickStarts/Python

In [0]:
import requests
from pprint import pprint
from IPython.display import HTML

In [0]:
#@title init api config
api_key = "" #@param {type:"string"}
service_region = "westus"
language = 'ja-JP'

In [0]:
text_analytics_base_url = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/"

#### sentiment api

In [0]:
sentiment_api_url = text_analytics_base_url + "sentiment"


In [0]:
documents = {'documents' : [
  {'id': '1', 'language': 'ja', 'text': '外国はここよりずっと遠いから友達の置いてゆく自転車'},
  {'id': '2', 'language': 'ja', 'text': 'Smoking kills. Smokers die younger. Smoking harms you and others.'},  
  {'id': '3', 'language': 'ja', 'text': '携帯のデフォルトのアラームが鳴り、もう一度鳴り、あなたは起きる。'}
]}

In [0]:
headers   = {"Ocp-Apim-Subscription-Key": api_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()

In [25]:
pprint(sentiments)

{'documents': [{'id': '1', 'score': 0.24311727285385132},
               {'id': '2', 'score': 0.5},
               {'id': '3', 'score': 0.409498929977417}],
 'errors': []}
